# Results Analysis on JudgeBench

## Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## Analysis

In [ ]:
# Define file paths for evaluation results
file_paths = {
    "Single": ["single_1.csv", "single_2.csv"],
    "Parallel": ["parallel_1.csv", "parallel_2.csv"],
    "Cooperative": ["cooperative_1.csv", "cooperative_2.csv"],
    "Competitive": ["competitive_1.csv", "competitive_2.csv"]
}

# Load evaluation results
data = {}
for system, paths in file_paths.items():
    df1 = pd.read_csv(f"Results/{paths[0]}")
    df2 = pd.read_csv(f"Results/{paths[1]}")
    data[system] = {"_1": df1, "_2": df2}

# Collect evaluation metrics
results = []

for system, dfs in data.items():
    df1, df2 = dfs["_1"], dfs["_2"]

    # Calculate accuracy (only valid A/B decisions)
    acc_1 = df1[df1["system_decision"].isin(["A", "B"])]["is_correct"].mean()
    acc_2 = df2[df2["system_decision"].isin(["A", "B"])]["is_correct"].mean()
    acc_avg = (acc_1 + acc_2) / 2

    # Calculate proportion of "X" responses
    x_1 = (df1["system_decision"] == "X").mean()
    x_2 = (df2["system_decision"] == "X").mean()

    # Consistency check (only valid A/B in both versions)
    valid_mask = df1["system_decision"].isin(["A", "B"]) & df2["system_decision"].isin(["A", "B"])
    s1 = df1[valid_mask]["system_decision"].reset_index(drop=True)
    s2 = df2[valid_mask]["system_decision"].reset_index(drop=True)
    consistent = s1 == s2
    inconsistency_rate = 1 - consistent.mean()

    # Store results
    results.append({
        "System": system,
        "Accuracy (normal)": f"{round(acc_1 * 100, 1)}%",
        "Accuracy (swapped)": f"{round(acc_2 * 100, 1)}%",
        "Accuracy (avg)": f"{round(acc_avg * 100, 1)}%",
        "Inconsistency": f"{round(inconsistency_rate * 100, 1)}%",
        "X rate (normal)": f"{round(x_1 * 100, 1)}%",
        "X rate (swapped)": f"{round(x_2 * 100, 1)}%"
    })

# Display summary
summary_df = pd.DataFrame(results)
display(summary_df)

In [ ]:
# Define German labels for the evaluation systems
system_labels_de = ["Einzeln", "Parallel", "Kooperativ", "Kompetitiv"]

# Extract accuracy values from summary DataFrame
acc_normal = summary_df["Accuracy (normal)"].str.replace("%", "").astype(float)
acc_swapped = summary_df["Accuracy (swapped)"].str.replace("%", "").astype(float)
acc_avg = summary_df["Accuracy (avg)"].str.replace("%", "").astype(float)

# Define bar positions
x = np.arange(len(system_labels_de))
bar_width = 0.25

plt.figure(figsize=(10, 4))

# Plot grouped bars
bars1 = plt.bar(x - bar_width, acc_normal, width=bar_width, label="Normal")
bars2 = plt.bar(x, acc_swapped, width=bar_width, label="Vertauscht")
bars3 = plt.bar(x + bar_width, acc_avg, width=bar_width, label="Durchschnitt")

# Labeling
def add_labels(bars):
    for bar in bars:
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            height + 1.5,
            f"{height:.1f}%",
            ha="center",
            va="bottom",
            fontsize=10
        )

add_labels(bars1)
add_labels(bars2)
add_labels(bars3)

# Axis labels and ticks
plt.xticks(x, system_labels_de, fontsize=11)
plt.yticks(np.arange(0, 101, 10), fontsize=11)
plt.ylim(0, 101)
plt.ylabel("Accuracy (%)", fontsize=12)

# Legend and layout
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Define German labels for the evaluation systems
system_labels_de = ["Einzeln", "Parallel", "Kooperativ", "Kompetitiv"]

# Extract inconsistency values
inconsistency = summary_df["Inconsistency"].str.replace("%", "").astype(float)

# Define color
blue = plt.get_cmap("tab10")(0)

plt.figure(figsize=(10, 4))
bars = plt.bar(system_labels_de, inconsistency, color=blue)

# Labeling
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height + 1.5,
        f"{height:.1f}%",
        ha="center",
        va="bottom",
        fontsize=10
    )

# Axis configuration
plt.ylim(0, 100)
plt.yticks(np.arange(0, 101, 10), fontsize=11)
plt.xticks(fontsize=11)
plt.ylabel("Inkonsistenz (%)", fontsize=12)

# Grid and layout
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Prepare bias data
bias_counts = []

# Mapping to German labels
system_labels_de = {
    "Single": "Einzeln",
    "Parallel": "Parallel",
    "Cooperative": "Kooperativ",
    "Competitive": "Kompetitiv"
}

# Filter valid decisions, count choices and store results
for system, dfs in data.items():
    for version in ["_1", "_2"]:
        df = dfs[version]
        valid = df[df["system_decision"].isin(["A", "B"])]
        count_A = (valid["system_decision"] == "A").sum()
        count_B = (valid["system_decision"] == "B").sum()
        total = count_A + count_B

        bias_counts.append({
            "System": system_labels_de[system],
            "Position": "Normal" if version == "_1" else "Vertauscht",
            "A gewählt (%)": round(100 * count_A / total, 1),
            "B gewählt (%)": round(100 * count_B / total, 1)
        })

# Create DataFrame
bias_df = pd.DataFrame(bias_counts)

# Prepare heatmap data
heatmap_df = bias_df.set_index(["System", "Position"])[["A gewählt (%)", "B gewählt (%)"]]

# Define specific row order
heatmap_df = heatmap_df.reindex([
    ("Einzeln", "Normal"), ("Einzeln", "Vertauscht"),
    ("Parallel", "Normal"), ("Parallel", "Vertauscht"),
    ("Kooperativ", "Normal"), ("Kooperativ", "Vertauscht"),
    ("Kompetitiv", "Normal"), ("Kompetitiv", "Vertauscht")
])

# Plot heatmap
plt.figure(figsize=(8, 4))
ax = sns.heatmap(
    heatmap_df,
    annot=True,
    fmt=".1f",
    cmap="Blues",
    cbar_kws={"label": "Prozent"},
    linewidths=0.5,
    linecolor="white"
)

# Add horizontal lines between systems
for i in range(2, len(heatmap_df), 2):
    ax.hlines(i, *ax.get_xlim(), colors="black", linewidth=1)

plt.ylabel("")
plt.tight_layout()
plt.show()